<a href="https://colab.research.google.com/github/CDCapobianco/Paintings-Classifier-AI/blob/main/Paintings_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Kaggle Dataset**

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!rm -r ~/.kaggle
! mkdir ~/.kaggle
!cp /content/gdrive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [3]:
! kaggle datasets download thedownhill/art-images-drawings-painting-sculpture-engraving

 99% 578M/583M [00:19<00:00, 41.8MB/s]
100% 583M/583M [00:19<00:00, 31.5MB/s]


In [4]:
import zipfile
with zipfile.ZipFile("/content/art-images-drawings-painting-sculpture-engraving.zip","r") as zip_ref:
  zip_ref.extractall("/content")


In [5]:
!cp -r /content/musemart/dataset_updated/training_set /content/dataset/dataset_updated
!cp -r /content/musemart/dataset_updated/validation_set /content/dataset/dataset_updated
!rm -r /content/musemart

In [6]:
!cp -r /content/dataset/dataset_updated/training_set/drawings /content/dataset/dataset_updated
!cp -r /content/dataset/dataset_updated/training_set/engraving /content/dataset/dataset_updated
!cp -r /content/dataset/dataset_updated/training_set/iconography /content/dataset/dataset_updated
!cp -r /content/dataset/dataset_updated/training_set/painting /content/dataset/dataset_updated
!cp -r /content/dataset/dataset_updated/training_set/sculpture /content/dataset/dataset_updated

In [7]:
!cp -r /content/dataset/dataset_updated/validation_set/drawings /content/dataset/dataset_updated
!cp -r /content/dataset/dataset_updated/validation_set/engraving /content/dataset/dataset_updated
!cp -r /content/dataset/dataset_updated/validation_set/iconography /content/dataset/dataset_updated
!cp -r /content/dataset/dataset_updated/validation_set/painting /content/dataset/dataset_updated
!cp -r /content/dataset/dataset_updated/validation_set/sculpture /content/dataset/dataset_updated

In [8]:
!rm -r /content/dataset/dataset_updated/training_set
!rm -r /content/dataset/dataset_updated/validation_set

# **Data Cleaning**

In [9]:
!pip install keras_adamw

     |████████████████████████████████| 462 kB 6.6 MB/s 


In [10]:
import os

In [11]:
IMAGES_PATH = "/content/dataset/dataset_updated"

In [12]:
def get_files_number(path=IMAGES_PATH):
  total = 0
  for root,dirs,files in os.walk(path):
    total+=len(files)
  return total

In [13]:
#rimuoviamo le immagini corrotte
from pathlib import Path
import imghdr

data_dir = IMAGES_PATH
image_extensions = [".png", ".jpg"]

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
            os.remove(filepath)
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")
            os.remove(filepath)

/content/dataset/dataset_updated/sculpture/107.jpg is not an image
/content/dataset/dataset_updated/sculpture/170.jpg is not an image
/content/dataset/dataset_updated/sculpture/98.jpg is not an image
/content/dataset/dataset_updated/sculpture/200.jpg is not an image
/content/dataset/dataset_updated/sculpture/190 18.59.45.jpg is not an image
/content/dataset/dataset_updated/sculpture/40.jpg is not an image
/content/dataset/dataset_updated/sculpture/329.jpg is not an image
/content/dataset/dataset_updated/sculpture/3.jpg is not an image
/content/dataset/dataset_updated/sculpture/190.jpg is not an image
/content/dataset/dataset_updated/sculpture/261.jpg is not an image
/content/dataset/dataset_updated/sculpture/3 18.59.45.jpg is not an image
/content/dataset/dataset_updated/sculpture/138.jpg is not an image
/content/dataset/dataset_updated/sculpture/106.jpg is not an image
/content/dataset/dataset_updated/sculpture/168.jpg is not an image
/content/dataset/dataset_updated/sculpture/374.jpg

In [14]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
STEPS_PER_EPOCH = 8
BATCH_SIZE = 32

In [15]:
import tensorflow as tf

In [16]:
os.environ["TF_KERAS"] = '1'
from keras_adamw import AdamW


train_set = tf.keras.utils.image_dataset_from_directory(
    IMAGES_PATH,
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size = (IMG_WIDTH,IMG_HEIGHT),
    batch_size = 32
)

validation_set = tf.keras.utils.image_dataset_from_directory(
    IMAGES_PATH,
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    image_size = (IMG_WIDTH,IMG_HEIGHT),
    batch_size = 32
)


Found 8741 files belonging to 5 classes.
Using 6993 files for training.
Found 8741 files belonging to 5 classes.
Using 1748 files for validation.


In [17]:
num_classes = train_set.class_names

In [18]:
type(train_set)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [19]:
def split_dataset(dataset: tf.data.Dataset, validation_data_fraction: float):

    validation_data_percent = round(validation_data_fraction * 100)
    if not (0 <= validation_data_percent <= 100):
        raise ValueError("validation data fraction must be ∈ [0,1]")

    dataset = dataset.enumerate() #supponendo validation_data_fraction = 0.3 
    train_dataset = dataset.filter(lambda f, data: f % 100 > validation_data_percent) #ogni 100 elementi, prende gli ultimi 70
    validation_dataset = dataset.filter(lambda f, data: f % 100 <= validation_data_percent) #ogni 100 elementi, prende i primi 30

    # rimuoviamo enumerazione
    train_dataset = train_dataset.map(lambda f, data: data)
    validation_dataset = validation_dataset.map(lambda f, data: data)

    return train_dataset, validation_dataset

In [20]:
train_set,test_set = split_dataset(train_set,0.2)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_set = train_set.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
validation_set = validation_set.cache().prefetch(buffer_size=AUTOTUNE)
test_set = test_set.cache().prefetch(buffer_size=AUTOTUNE)

# **Training and Testing**

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255,input_shape = [IMG_HEIGHT,IMG_WIDTH,3]),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(len(num_classes))
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 127, 127, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                    

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=["accuracy"])

In [ ]:
def learning_rate_exponential(epoch):
  return 0.1*0.1**(epoch/20)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.25,patience=5)

In [ ]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=20,verbose=1)

In [ ]:
history = model.fit(train_set,epochs=35,validation_data=(validation_set),callbacks=[lr_scheduler])

Epoch 1/35


In [ ]:
prediction = model.evaluate(test_set)

In [ ]:
import numpy as np

y_pred = []
y_true = []

for image_batch, label_batch in test_set:
   y_true.append(label_batch)
   preds = model.predict(image_batch)
   y_pred.append(np.argmax(preds, axis = - 1))

correct_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)

In [ ]:
confusion_matrix = tf.math.confusion_matrix(correct_labels,predicted_labels)

In [ ]:
confusion_matrix

In [ ]:
import seaborn as sns 
import pandas as pd 
from matplotlib import pyplot as plt

cm = pd.DataFrame(confusion_matrix.numpy(), # use .numpy(), because now confusion is tensor
               range(5),range(5))

plt.figure(figsize = (10,10))
sns.heatmap(cm, annot=True, annot_kws={"size": 12})
plt.show()

# **Transfer Learning with MobileNetV2**

In [36]:
import tensorflow as tf

In [54]:
train_set = tf.keras.utils.image_dataset_from_directory(
    IMAGES_PATH,
    validation_split = 0.2,
    subset = "training",
    seed = 25,
    image_size = (224,224),
    batch_size = 32
)

validation_set = tf.keras.utils.image_dataset_from_directory(
    IMAGES_PATH,
    validation_split = 0.2,
    subset = "validation",
    seed = 25,
    image_size = (224,224),
    batch_size = 32
)

Found 8741 files belonging to 5 classes.
Using 6993 files for training.
Found 8741 files belonging to 5 classes.
Using 1748 files for validation.


In [55]:
validation_cardinality = tf.data.experimental.cardinality(validation_set)
test_set = validation_set.take(validation_cardinality // 5)
validation_set = validation_set.skip(validation_cardinality // 5)

In [56]:
pretrained_model = tf.keras.applications.MobileNetV2(weights='imagenet',include_top=False,input_shape=[224,224,3])

In [57]:
AUTOTUNE  = tf.data.AUTOTUNE

train_set = train_set.prefetch(buffer_size = AUTOTUNE)
test_set = test_set.prefetch(buffer_size = AUTOTUNE)
validation_set = validation_set.prefetch(buffer_size = AUTOTUNE)

In [58]:
for layer in pretrained_model.layers:
  layer.trainable=False

In [59]:
preprocessing_layer = tf.keras.layers.Rescaling(1./127.5,offset=-1,input_shape=[224,224,3])
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(5)

inputs = tf.keras.Input(shape=(224, 224, 3))
x = preprocessing_layer(inputs)
x = pretrained_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dense(128,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [60]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(patience=2,factor=0.5,monitor="val_loss")

In [64]:
earlystopping = tf.keras.callbacks.EarlyStopping(patience=5,monitor="val_accuracy",verbose=1,restore_best_weights=True)

In [65]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [66]:
history = model.fit(train_set,validation_data=(validation_set),epochs=25,callbacks=[lr_scheduler,earlystopping])

Epoch 1/25
219/219 [==============================] - 29s 116ms/step - loss: 0.2371 - accuracy: 0.9175 - val_loss: 0.2839 - val_accuracy: 0.9076 - lr: 0.0100
Epoch 2/25
219/219 [==============================] - 25s 112ms/step - loss: 0.2196 - accuracy: 0.9234 - val_loss: 0.3063 - val_accuracy: 0.9119 - lr: 0.0100
Epoch 3/25
219/219 [==============================] - 25s 110ms/step - loss: 0.2322 - accuracy: 0.9139 - val_loss: 0.2754 - val_accuracy: 0.9191 - lr: 0.0100
Epoch 4/25
219/219 [==============================] - 25s 110ms/step - loss: 0.2066 - accuracy: 0.9239 - val_loss: 0.2877 - val_accuracy: 0.9205 - lr: 0.0100
Epoch 5/25
219/219 [==============================] - 24s 110ms/step - loss: 0.1925 - accuracy: 0.9271 - val_loss: 0.2675 - val_accuracy: 0.9133 - lr: 0.0100
Epoch 6/25
219/219 [==============================] - 25s 110ms/step - loss: 0.1942 - accuracy: 0.9278 - val_loss: 0.3147 - val_accuracy: 0.9040 - lr: 0.0100
Epoch 7/25
219/219 [==============================] 

In [67]:
model.save_weights('tl_weights.h5')

In [68]:
model.evaluate(test_set)

11/11 [==============================] - 2s 95ms/step - loss: 0.4152 - accuracy: 0.9176


[0.41517049074172974, 0.9176136255264282]

In [69]:
fine_tune_at = 100

for layer in pretrained_model.layers[fine_tune_at:]:
  layer.trainable=True

In [70]:
model.optimizer.learning_rate.assign(model.optimizer.learning_rate / 10)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=0.00012499999>

In [71]:
history_fine = model.fit(train_set,validation_data=(validation_set),epochs=10)

Epoch 1/10
219/219 [==============================] - 25s 111ms/step - loss: 0.1177 - accuracy: 0.9582 - val_loss: 0.2782 - val_accuracy: 0.9291
Epoch 2/10
219/219 [==============================] - 25s 111ms/step - loss: 0.1098 - accuracy: 0.9591 - val_loss: 0.2512 - val_accuracy: 0.9327
Epoch 3/10
219/219 [==============================] - 25s 111ms/step - loss: 0.0975 - accuracy: 0.9628 - val_loss: 0.2851 - val_accuracy: 0.9284
Epoch 4/10
219/219 [==============================] - 25s 110ms/step - loss: 0.0938 - accuracy: 0.9658 - val_loss: 0.3024 - val_accuracy: 0.9305
Epoch 5/10
219/219 [==============================] - 25s 111ms/step - loss: 0.0855 - accuracy: 0.9653 - val_loss: 0.2823 - val_accuracy: 0.9362
Epoch 6/10
219/219 [==============================] - 33s 144ms/step - loss: 0.0869 - accuracy: 0.9663 - val_loss: 0.2460 - val_accuracy: 0.9384
Epoch 7/10
219/219 [==============================] - 25s 110ms/step - loss: 0.0858 - accuracy: 0.9651 - val_loss: 0.3104 - val_ac

In [73]:
model.evaluate(test_set)

11/11 [==============================] - 2s 94ms/step - loss: 0.4609 - accuracy: 0.9347


[0.4608772397041321, 0.9346590638160706]